# Add Embeddings to an Existing Table


In this example we will extend an existing table with embeddings computed from a pre-trained model.

- Write an initial table containing a single column of image URLs.
- Write a new table containing the input URLs and the embeddings computed from a pre-trained model.
- Apply dimensionality reduction to the extended table to get a final table containing the URLs, the embeddings, and the reduced embeddings.

In [1]:
import tlc

## Load the initial table

We load the previously written COCO128 table (see [create-table-from-coco.ipynb](../1-create-tables/create-table-from-coco.ipynb)).

In [2]:
table = tlc.Table.from_names("initial", "COCO128", "3LC Tutorials")

## Extend the table with embeddings from a pre-trained model

We will use the ViT model pre-trained on ImageNet to compute embeddings for the images in the table.
A benefit of using this model is that meaningful embeddings can be extracted easily using the `last_hidden_state` attribute of the model output.

In [3]:
from tools.embeddings import add_embeddings_to_table
from PIL import Image

table.map(lambda x: Image.open(x["image"]).convert("RGB"))
extended_table = add_embeddings_to_table(table)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Extracting embeddings:   0%|          | 0/32 [00:00<?, ?it/s]c:\Project\notebook-examples\.venv\Lib\site-packages\transformers\models\vit\modeling_vit.py:253: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(
Extracting embeddings: 100%|██████████| 32/32 [00:02<00:00, 13.62it/s]


## Reduce the embeddings to 2 dimensions

Finally we reduce the embedding-column to 2 dimensions using UMAP. The result is a table containing the URLs, the embeddings, and the reduced embeddings.

In [4]:
reduced_table = tlc.reduce_embeddings(
    extended_table,
    method="pacmap",
    n_components=2,
    retain_source_embedding_column=False,
)

print(reduced_table.table_rows[0].keys()) # The row-view of the reduced table contains both the embeddings and the reduced embeddings

3lc: Fitting PaCMAP model on column 'embedding' for table ../with_embeddings
3lc: Transforming column 'embedding' in table ../with_embeddings with PaCMAP model ./reduced_0000.pkl
dict_keys(['image_id', 'image', 'bbs', 'width', 'height', 'weight', 'embedding_pacmap'])


In [5]:
reduced_table.model_url

Url('relative://reduced_0000.pkl')